## Pair Programming ETL III Transformacion II
### Modulo 2 Sprint 2
#### Lola Rubio y Judith Mellidez

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity
import requests
import pandas as pd
import requests
import ast
from datetime import datetime, timedelta

In [9]:
df =pd.read_csv('attacks_limpieza_completa.csv',index_col = 0) 
df.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,...,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,...,1,0,0,1,0,0,3,7,7,3.804860


In [36]:
#Definimos producto y diccionario. 
producto= 'meteo'
dicc_paises= {'USA':[-100.445882,39.7837304],
            'Australia':[134.755,-24.7761086],
            'South Africa':[24.991639,-28.8166236],
            'New Zealand':[172.8344077,-41.5000831],
            'Papua New Guinea':[144.2489081,-5.6816069]}

In [53]:
#Definimos la clase con los atributos y los metodos

class Transformacion_datos:
    def __init__(self,lon, lat, pais):#El _init_ es el Constructor, dentro del mismo defino los atributos y los inicializo 
                                     #con los valores que al Constructor al llamarle.
        self.lon = lon
        self.lat = lat
        self.pais = pais

    def llamada_API(self, producto): #Realizamos la llamada a la url mediante la API, los datos que obtenemos los devolvemos en un DF.
        
        self.producto=producto #definimos el atributo de la variable producto par aintroducirlo en la url.
        df_vacio= pd.DataFrame()

        url = f'http://www.7timer.info/bin/api.pl?lon=-{self.lon}&lat={self.lat}&product={self.producto}&output=json'

        response = requests.get(url=url)
        codigo_estado =response.status_code 
        razon_estado = response.reason
        
        if codigo_estado == 200:
            print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar usario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
        else:
            print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)

        df_paises =pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
        df_vacio= pd.concat([df_vacio, df_paises],ignore_index = True ,axis =0) #Me anade la respuesta para una lon, lat, pais, en especifico.

        #Añado a todas las lineas de mi df las columnas con el valor del atributo. 
        #Las anado debajo porque si las anado despues del df_vacio, solo me genera una linea correpondiente con tres columnas y al llegar al 
        #concact se carga el mas pequeno que seria la linea creada y nos devuelve solo el df_paises. ### PREGUNTAR A ANA POR QUE!!!
        df_vacio['lon']= self.lon
        df_vacio['lat']= self.lat   
        df_vacio['pais']= self.pais
        
        return df_vacio

    def desempaquetar (self,df_climatico): #Me convierte una columna en muchas columnas. 
        #df_climatico['rh_profile'] = df_climatico['rh_profile'].apply(ast.literal_eval)
        rh = df_climatico['rh_profile'].apply(pd.Series)

        for i in range(len(rh.columns)): 
            name = "rh_new" + str(rh[i].apply(pd.Series)['layer'][0])
            value = list(rh[i].apply(pd.Series)["rh"] )
            df_climatico.insert(i, name, value)

            #df_climatico['wind_profile'] = df_climatico['wind_profile'].apply(ast.literal_eval)   
        
        
        wind = df_climatico['wind_profile'].apply(pd.Series)

        for i in range(len(wind.columns)): 
            name = 'wind_direct' + str(wind[i].apply(pd.Series)['layer'][0])
            values = list(wind[i].apply(pd.Series)['direction'] )
            df_climatico.insert(i, name, values)

        for i in range(len(wind.columns)): 
            name2 ='wind_speed' + str(wind[i].apply(pd.Series)['layer'][0])
            values2 = list(wind[i].apply(pd.Series)['speed'])
            df_climatico.insert(i, name2, values2)

        return df_climatico
        

In [54]:
#Ejemplo de uso para un solo dato, dado que no hemos creado el loop dentro de la clase. 
#Nos daba error y hemos decidido crearlo para un solo dato y llamar a la clase en 5 ocasiones para que nos de la informacion que estamos buscando. 
#nos da error si metemos todos los paises, por eso hemos probado con uno, y viendo que nos resultaba, hemos hecho el loop.
api = Transformacion_datos('-100.445882','39.7837304','usa')

In [55]:
#Como la llamada a API me devuelve un df, guardamos este df como df_api.
df_api = api.llamada_API(producto)
df_api.head(2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat,pais
0,3,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 150, 'speed':...",16,6,4,1024,none,0,0,110,2,-100.445882,39.7837304,usa
1,6,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",15,6,6,1025,none,0,0,220,2,-100.445882,39.7837304,usa


In [33]:
df_vacio = api.desempaquetar(df_api)
df_vacio.head(2)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat,pais
0,2,2,3,3,4,5,5,6,6,6,...,4,1024,none,0,0,110,2,-100.445882,39.7837304,usa
1,2,3,4,3,3,3,4,6,6,6,...,6,1025,none,0,0,220,2,-100.445882,39.7837304,usa


In [57]:
#Creamos un loop para llamarlo 5 veces, 'dicc_paises' (donde tenemos almacenados los 5 paises con sus lon y lat) y que nos junte los datos.
df_api_paises = pd.DataFrame()

for k,v in dicc_paises.items():
    apipais = Transformacion_datos(v[0],v[1],k)
    print(df_api_paises)
    df_apipais = apipais.llamada_API(producto)
    df_vacioapipais = apipais.desempaquetar(df_apipais)
    print(df_vacioapipais)
    df_api_paises = pd.concat([df_api_paises, df_vacioapipais], axis=0, ignore_index=True)


La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


In [51]:
df_api_paises.sample(5)

,wind_speed950mb,wind_speed900mb,wind_speed850mb,wind_speed800mb,wind_speed750mb,wind_speed700mb,wind_speed650mb,wind_speed600mb,wind_speed550mb,wind_speed500mb,...,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,lon,lat,pais
79,4,4,4,4,4,4,4,4,5,5,...,11,1015,rain,2,0,100,4,134.755000,-24.776109,Australia
305,5,5,5,5,5,3,3,3,4,5,...,12,1012,rain,3,0,80,4,144.248908,-5.681607,Papua New Guinea
197,4,4,3,4,4,4,4,5,5,6,...,14,1000,rain,2,0,300,3,172.834408,-41.500083,New Zealand
100,3,3,3,3,3,3,3,3,3,3,...,11,1013,rain,3,0,80,3,134.755000,-24.776109,Australia
278,4,4,4,4,3,2,2,2,3,4,...,10,1011,none,2,0,75,3,144.248908,-5.681607,Papua New Guinea
